In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read the data called 'df' and checking its head
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [3]:
df.shape

(10000, 14)

In [4]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [5]:
# description of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [6]:
df.drop('stab', axis=1, inplace=True)
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [7]:
#splitting features and target variable
X = df.drop('stabf', axis=1)
y = df['stabf']

In [8]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2 , random_state= 1 )
print('X_train shape: {}'.format(x_train.shape))
print('y_train shape: {}'.format(y_train.shape))
print('X_test shape: {}'.format(x_test.shape))
print('y_test shape: {}'.format(y_test.shape))

X_train shape: (8000, 12)
y_train shape: (8000,)
X_test shape: (2000, 12)
y_test shape: (2000,)


In [9]:
#Scaling dataset using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
#use standard scaler to transform train and test
x_train_scaler = scaler.fit_transform(x_train)
x_test_scaler = scaler.transform(x_test)

In [11]:
#put the scaled sets into a daataframe

x_train_scaled = pd.DataFrame(x_train_scaler, columns = x_train.columns)
x_test_scaled = pd.DataFrame(x_test_scaler, columns = x_test.columns)

Question # 1

In [13]:
#f1_score
Tp = 255
Tn = 20
Fp = 1380
Fn = 45
precision = Tp / (Tp + Fp)
recall= Tp / (Tp + Fn)
f1_score=  2*((precision* recall)/(precision+recall))

round(f1_score,4)

0.2636

#### RandomForestClassifier

In [14]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(random_state = 1)
random_forest.fit(x_train_scaled, y_train)

RandomForestClassifier(random_state=1)

In [15]:
#make predictions on test set
randf_pred = random_forest.predict(x_test_scaled)

Model Performance for RandomForestClassifier

In [23]:
#model accuracy
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
RFC_accuracy = accuracy_score(y_true=y_test, y_pred=randf_pred)
print('Accuracy: {}'.format(RFC_accuracy,4))
#print('Accuracy: {}'.format(round(RFC_accuracy*100), 4))

Accuracy: 0.929


ExtraTreeClassifier

In [22]:
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(random_state = 1)

#fit on the train set
etc.fit(x_train_scaled, y_train)

#predict on test set
etc_pred = etc.predict(x_test_scaled)

In [23]:
#model accuracy
etc_accuracy = accuracy_score(y_test, etc_pred)
print('Accuracy: {}'.format(round(etc_accuracy*100), 2))

Accuracy: 93


In [24]:
#classification report
print('Classification Report:\n', classification_report(y_test,etc_pred, digits =4))

Classification Report:
               precision    recall  f1-score   support

      stable     0.9410    0.8511    0.8938       712
    unstable     0.9218    0.9705    0.9455      1288

    accuracy                         0.9280      2000
   macro avg     0.9314    0.9108    0.9197      2000
weighted avg     0.9287    0.9280    0.9271      2000



LightGBM Classifier

In [25]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(random_state = 1)

#fit on train set
lgbm.fit(x_train_scaled, y_train)

#predict on test set
lgbm_pred = lgbm.predict(x_test_scaled)

In [26]:
# model accuracy
lgbm_accuracy = accuracy_score(y_test, lgbm_pred)
print('Accuracy: {}'.format(round(lgbm_accuracy*100), 2))

Accuracy: 94


In [27]:
#classification report
print('Classification Report:\n', classification_report(y_test,lgbm_pred, digits =4))

Classification Report:
               precision    recall  f1-score   support

      stable     0.9297    0.8919    0.9104       712
    unstable     0.9415    0.9627    0.9520      1288

    accuracy                         0.9375      2000
   macro avg     0.9356    0.9273    0.9312      2000
weighted avg     0.9373    0.9375    0.9372      2000



XGBoost Classifier

In [24]:
from xgboost import XGBClassifier

xbc= XGBClassifier(random_state = 1)

#fit on train set
xbc.fit(x_train_scaled, y_train)

XGBClassifier(random_state=1)

In [25]:
#predict on test set
xbc_pred = xbc.predict(x_test_scaled)

In [27]:
#Accuracy
xbg_accuracy = accuracy_score(y_test, xbc_pred)
print('Accuracy: {}'.format(xbg_accuracy))
#print('Accuracy: {}'.format(round(xbg_accuracy*100), 2))

Accuracy: 0.9195


LightGBM Classifier

In [30]:
from lightgbm import LGBMClassifier

lgbm= LGBMClassifier(random_state = 1)

#fit on train set
lgbm.fit(x_train_scaled, y_train)

LGBMClassifier(random_state=1)

In [31]:
lgbm_pred = lgbm.predict(x_test_scaled)

In [32]:
#Lightgbm Accuracy
accuracy = accuracy_score(y_test,lgbm_pred)
print(f"Accuracy: {round(accuracy,4)}")

Accuracy: 0.9375
